## Offensive Language Identification

This is the task 'a' in Task 6: OffensEval: Identifying and Categorizing Offensive Language in Social Media in SemEval 2019. We have explored how a RNN with attention + cyclic learning rate can be used in Offensive language identification. 

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split

In [2]:
SEQ_LEN = 100  # maBaselinegic number - length to truncate sequences of words

Loading the training and testing files and Changing case of the tweets to lower case, since the embedding model only has lower case words

In [3]:
train_df = pd.read_csv("Data/offenseval-training-v1.tsv", sep='\t')
test_df = pd.read_csv("Data/test.tsv")

train_df["tweet"] = train_df["tweet"].str.lower()
test_df["tweet"] = test_df["tweet"].str.lower()

Cleaning the puncutation marks

In [4]:
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

def clean_text(x):

    x = str(x)
    for punct in puncts:
        x = x.replace(punct, f' {punct} ')
    return x
test_df

train_df["tweet"] = train_df["tweet"].apply(lambda x: clean_text(x))
test_df["tweet"] = test_df["tweet"].apply(lambda x: clean_text(x))

In [5]:
train_df, val_df = train_test_split(train_df, test_size=0.1)
train_df[:5]

,id,tweet,subtask_a,subtask_b,subtask_c
12030,51937,@ user damn is she ugly ! ! ! !,1,TIN,IND
11682,78293,@ user no way ! she is !,0,NaN,NaN
9235,95376,@ user @ user @ user @ user @ user @ use...,0,NaN,NaN
10971,16502,# mogg : # jacobreesmogg calls for @ user ...,1,TIN,IND
5236,91767,@ user @ user @ user @ user @ user i thin...,1,TIN,IND


In [6]:
#minor eda: average question length (in words) is 22  , majority are under 18 words
train_df.tweet.str.split().str.len().describe()

count    11916.000000
mean        29.273582
std         20.435321
min          3.000000
25%         14.000000
50%         24.000000
75%         41.000000
max        163.000000
Name: tweet, dtype: float64

Loading word vectors

In [7]:
# embedding setup
# Source https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html
# 
embeddings_index = {}
f = open('/data/glove/glove.840B.300d.txt')
# f = open('../input/embeddings/paragram_300_sl999/paragram_300_sl999.txt')
for line in tqdm(f):
    values = line.split(" ")
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

2196017it [01:32, 23705.50it/s]

Found 2196016 word vectors.


In [8]:
import re
_WORD_SPLIT = re.compile("([.,!?\"':;)(])")
_DIGIT_RE = re.compile(br"\d")
STOP_WORDS = "\" \' [ ] . , ! : ; ?".split(" ")
def basic_tokenizer(sentence):
    """Very basic tokenizer: split the sentence into a list of tokens."""
    words = []
    for space_separated_fragment in sentence.strip().split():
        words.extend(_WORD_SPLIT.split(space_separated_fragment))
        # return [w.lower() for w in words if w not in stop_words and w != '' and w != ' ']
    return [w.lower() for w in words if w != '' and w != ' ']

In [9]:
# Convert values to embeddings
def text_to_array(text):
    empyt_emb = np.zeros(300)
    text = basic_tokenizer(text[:-1])[:SEQ_LEN]
    embeds = [embeddings_index.get(x, empyt_emb) for x in text]
    embeds+= [empyt_emb] * (SEQ_LEN - len(embeds))
    return np.array(embeds)

# train_vects = [text_to_array(X_text) for X_text in tqdm(train_df["question_text"])]
val_vects = np.array([text_to_array(X_text) for X_text in tqdm(val_df["tweet"])])
val_y = np.array(val_df["subtask_a"])

100%|██████████| 1324/1324 [00:00<00:00, 9959.43it/s]


In [10]:
# Data providers
batch_size = 32

def batch_gen(train_df):
    n_batches = math.ceil(len(train_df) / batch_size)
    while True: 
        train_df = train_df.sample(frac=1.)  # Shuffle the data.
        for i in range(n_batches):
            texts = train_df.iloc[i*batch_size:(i+1)*batch_size, 1]
            text_arr = np.array([text_to_array(text) for text in texts])
            yield text_arr, np.array(train_df["subtask_a"][i*batch_size:(i+1)*batch_size])

## Training

In [11]:
from keras.models import Sequential,Model
from keras.layers import LSTM, Dense, Bidirectional, Input,Dropout

from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints

Implementation of the Attention Layer

In [15]:
# https://www.kaggle.com/qqgeogor/keras-lstm-attention-glove840b-lb-0-043
class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True
        
    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

Combinning everything together LSTM, LSTM + Attention

In [13]:
inp = Input(shape=(SEQ_LEN,300 ))
x = Bidirectional(LSTM(64, return_sequences=True))(inp)
x = Bidirectional(LSTM(64,return_sequences=True))(x)
x = Attention(SEQ_LEN)(x)
x = Dense(256, activation="relu")(x)
# x = Dropout(0.25)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


Training the model

In [14]:
mg = batch_gen(train_df)
model.fit_generator(mg, epochs=20,
                    steps_per_epoch=1000,
                    validation_data=(val_vects, val_y),
                    verbose=True)

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
1000/1000 [==============================] - 80s 80ms/step - loss: 0.4748 - acc: 0.7773 - val_loss: 0.4523 - val_acc: 0.8029
Epoch 2/20
1000/1000 [==============================] - 78s 78ms/step - loss: 0.3956 - acc: 0.8195 - val_loss: 0.5003 - val_acc: 0.7704
Epoch 3/20
1000/1000 [==============================] - 78s 78ms/step - loss: 0.3084 - acc: 0.8681 - val_loss: 0.7141 - val_acc: 0.7734
Epoch 4/20
1000/1000 [==============================] - 80s 80ms/step - loss: 0.1940 - acc: 0.9242 - val_loss: 0.9553 - val_acc: 0.7538
Epoch 5/20
1000/1000 [==============================] - 77s 77ms/step - loss: 0.1079 - acc: 0.9609 - val_loss: 1.3292 - val_acc: 0.7515
Epoch 6/20
1000/1000 [==============================] - 77s 77ms/step - loss: 0.0674 - acc: 0.9765 - val_loss: 1.3947 - val_acc: 0.7568
Epoch 7/20
1000/1000 [==============================] - 78s 78ms/step - loss: 0.0463 - acc: 0.9839 - val_loss: 1.4083 - val_acc: 0.7742


Predicting from the model

In [29]:
# prediction part
batch_size = 256
def batch_gen(test_df):
    n_batches = math.ceil(len(test_df) / batch_size)
    for i in range(n_batches):
        texts = test_df.iloc[i*batch_size:(i+1)*batch_size, 1]
        text_arr = np.array([text_to_array(text) for text in texts])
        yield text_arr


all_preds = []
for x in tqdm(batch_gen(test_df)):
    all_preds.extend(model.predict(x).flatten())

4it [00:01,  3.29it/s]


Converting the predictions in to integers

In [30]:
y_te = (np.array(all_preds) > 0.35).astype(np.int)

### Confusion Matrix

In [34]:
from sklearn.metrics import confusion_matrix

test_df['predictions'] = y_te
tn, fp, fn, tp = confusion_matrix(test_df["subtask_a"], test_df['predictions']).ravel()
(tn, fp, fn, tp)

(542, 78, 92, 148)

### Accuracy

In [35]:
from sklearn.metrics import accuracy_score
accuracy_score(test_df["subtask_a"], test_df['predictions'])

0.8023255813953488